# Thông Tin Sinh Viên
1. Họ tên: Võ Khoa Nam
2. Mssv: 19521877
3. Lớp: CS116.M12.KHCL

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Thêm một số thư viện
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Read Data

In [5]:
# Đọc file data bằng geopandas
data = pd.read_csv('/content/drive/MyDrive/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# Bỏ 3 cột đầu tiên do không ảnh hưởng đến output
data = data.drop(data.columns[[0, 1, 2]], axis = 1)

In [7]:
# Data sau khi bỏ 3 cột
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# OneHotEncoder

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
ct = ColumnTransformer(transformers = [('encode', OneHotEncoder(),['Geography','Gender'])],remainder = 'passthrough')
dt_trf  = ct.fit_transform(data)

df_tranform = pd.DataFrame(dt_trf)

In [10]:
df_tranform.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,1.0,0.0,619.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88,1.0
1,0.0,0.0,1.0,1.0,0.0,608.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58,0.0
2,1.0,0.0,0.0,1.0,0.0,502.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57,1.0
3,1.0,0.0,0.0,1.0,0.0,699.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63,0.0
4,0.0,0.0,1.0,1.0,0.0,850.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10,0.0


# Split Data

In [11]:
# Chia tập dữ liệu
X = df_tranform[df_tranform.columns[0:13]].to_numpy()
y = df_tranform[df_tranform.columns[-1]]

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

# Scaler Data

In [14]:
#Standar Scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [15]:
X_train = scaler.fit_transform(X_train)

# Built Neural Network

In [16]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [17]:
def Model():
  classifier = Sequential()

  classifier.add(Dense(6, activation = 'relu',input_dim = 13))
  classifier.add(Dense(6, activation = 'relu'))
  classifier.add(Dense(1, activation = 'sigmoid'))
  classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return classifier

In [18]:
model = Model()

In [19]:
# Fitting our model 
history = model.fit(X_train,y_train,epochs=32,batch_size=100)

Epoch 1/32
80/80 [==============================] - 1s 1ms/step - loss: 0.6275 - accuracy: 0.7015
Epoch 2/32
80/80 [==============================] - 0s 1ms/step - loss: 0.5448 - accuracy: 0.7966
Epoch 3/32
80/80 [==============================] - 0s 1ms/step - loss: 0.5054 - accuracy: 0.7983
Epoch 4/32
80/80 [==============================] - 0s 1ms/step - loss: 0.4788 - accuracy: 0.8018
Epoch 5/32
80/80 [==============================] - 0s 1ms/step - loss: 0.4589 - accuracy: 0.8059
Epoch 6/32
80/80 [==============================] - 0s 1ms/step - loss: 0.4428 - accuracy: 0.8089
Epoch 7/32
80/80 [==============================] - 0s 2ms/step - loss: 0.4294 - accuracy: 0.8104
Epoch 8/32
80/80 [==============================] - 0s 1ms/step - loss: 0.4185 - accuracy: 0.8126
Epoch 9/32
80/80 [==============================] - 0s 2ms/step - loss: 0.4095 - accuracy: 0.8151
Epoch 10/32
80/80 [==============================] - 0s 1ms/step - loss: 0.4012 - accuracy: 0.8185
Epoch 11/32
80/80 [

In [20]:
X_test_scale = scaler.transform(X_test)

In [21]:
# Hàm tích predict
def predicttest(model, X_test_scale, threshold=0.5):
  y_pred = model.predict(X_test_scale)
  pred = list()
  for i in range(len(y_pred)):
    if(y_pred[i] > threshold):
      pred.append(1)
    else:
      pred.append(0)
  return pred

In [22]:
predict = (predicttest(model, X_test_scale))

In [23]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(predict,y_test)
print('Accuracy is: ',round(acc,2)*100)

Accuracy is:  86.0


# Kfold

In [31]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [32]:
X_scale_data = scaler.transform(X)

In [33]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)

krkf = KerasClassifier(build_fn=Model, epochs=100, batch_size=32, verbose=0)

scores = cross_val_score(krkf, X_scale_data, y, scoring='accuracy', cv=kf)

In [34]:
scores

array([0.8695, 0.862 , 0.8685, 0.854 , 0.8565])

In [36]:
print('Accuracy is: ',round(scores.mean()*100,2))

Accuracy is:  86.21
